In [21]:
# 利用Keras实现基于cnn的手写识别
# https://zhuanlan.zhihu.com/p/33307255
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam

#输出数据类别
# 输入的图片是28*28像素的灰度图
num_classes=10
img_rows, img_cols = 28, 28
# 载入训练集，测试集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [22]:
# print(y_train[0])

In [23]:
# 会将100张RGB三通道的16*32彩色图表示为(100,16,32,3)，
# 第一个维度是样本维，表示样本的数目，
# 第二和第三个维度是高和宽，
# 最后一个维度是通道维，表示颜色通道数
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [24]:
# 把数据变成float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [25]:
# 把类别0-9变成2进制，方便训练
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [26]:
# 加入一个序贯模型
model = Sequential()

In [27]:
# 加入一个具有6个输出（也就是卷积通道）的2D卷积层，激活函数选用relu，padding为valid，过滤器的尺寸选用5*5像素窗口
# 加入一个尺寸为2*2的最大池化层
# 加入一个具有16个输出（也就是卷积通道）的2D卷积层，激活函数选用relu，padding为valid，过滤器的尺寸选用5*5像素窗口
# 加入一个尺寸为2*2的最大池化层
model.add(Conv2D(filters=6,activation='relu', input_shape=input_shape,padding='valid',kernel_size=(5,5)))
# add BatchNormalization
model.add(BatchNormalization(epsilon=1e-6,axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=16, activation='relu',padding='valid',kernel_size=(5,5)))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [28]:
#加入一个Flatten层
#按照Keras官方的解释：Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡。
#Flatten不影响batch的大小。
#加入一个有120个输出的全连接层，激活函数选用relu
#加入一个有84个输出的全连接层，激活函数选用relu
#加入一个有10个输出的输出层，激活函数选用softmax
model.add(Flatten())  
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [29]:
adam_lr = 0.0002
adam_beta_1 = 0.5

In [30]:
#组建工程并训练
#optimizer='sgd'
model.compile(optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),loss='categorical_crossentropy',metrics=['accuracy'])
# epochs=12
model.fit(x_train, y_train, batch_size=128, epochs=6, verbose=1, shuffle=True)  
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/6
60000/60000 [==============================] - 78s 1ms/step - loss: 0.4804 - acc: 0.8640
Epoch 2/6
60000/60000 [==============================] - 80s 1ms/step - loss: 0.1220 - acc: 0.9637
Epoch 3/6
60000/60000 [==============================] - 83s 1ms/step - loss: 0.0892 - acc: 0.9727
Epoch 4/6
60000/60000 [==============================] - 75s 1ms/step - loss: 0.0721 - acc: 0.9784
Epoch 5/6
60000/60000 [==============================] - 83s 1ms/step - loss: 0.0605 - acc: 0.9814
Epoch 6/6
10000/10000 [==============================] - 10s 955us/step
Test loss: 0.05509914552643895
Test accuracy: 0.9807


In [31]:
# add BatchNormalization
# Epoch 1/6
# 60000/60000 [==============================] - 86s 1ms/step - loss: 0.5034 - acc: 0.8493
# Epoch 2/6
# 60000/60000 [==============================] - 72s 1ms/step - loss: 0.1613 - acc: 0.9515
# Epoch 3/6
# 60000/60000 [==============================] - 72s 1ms/step - loss: 0.1151 - acc: 0.9654
# Epoch 4/6
# 60000/60000 [==============================] - 71s 1ms/step - loss: 0.0925 - acc: 0.9720
# Epoch 5/6
# 60000/60000 [==============================] - 73s 1ms/step - loss: 0.0789 - acc: 0.9765
# Epoch 6/6
# 60000/60000 [==============================] - 81s 1ms/step - loss: 0.0699 - acc: 0.9785
# 10000/10000 [==============================] - 9s 871us/step
# Test loss: 0.061645476794615385
# Test accuracy: 0.9794

In [32]:
#保存模型到文件
model.save('mnist_cnn_keras.h5')